In [ ]:
%cd /content
!git clone -b dev1 https://github.com/camenduru/PanoHead

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/PanoHead/resolve/main/ablation-trigridD-1-025000.pkl -d /content/PanoHead/models -o ablation-trigridD-1-025000.pkl
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/PanoHead/resolve/main/baseline-easy-khair-025000.pkl -d /content/PanoHead/models -o baseline-easy-khair-025000.pkl
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/PanoHead/resolve/main/easy-khair-180-gpc0.8-trans10-025000.pkl -d /content/PanoHead/models -o easy-khair-180-gpc0.8-trans10-025000.pkl

!pip install imgui glfw pyspng mrcfile ninja plyfile trimesh onnxruntime onnx

%cd /content
!git clone -b dev https://github.com/camenduru/3DDFA_V2
%cd /content/3DDFA_V2
!sh ./build.sh

!cp -rf /content/PanoHead/3DDFA_V2_cropping/test /content/3DDFA_V2
!cp /content/PanoHead/3DDFA_V2_cropping/dlib_kps.py /content/3DDFA_V2
!cp /content/PanoHead/3DDFA_V2_cropping/recrop_images.py /content/3DDFA_V2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/shape_predictor_68_face_landmarks/resolve/main/shape_predictor_68_face_landmarks.dat -d /content/3DDFA_V2 -o shape_predictor_68_face_landmarks.dat

!mkdir /content/in /content/stage /content/output

/content
Cloning into 'PanoHead'...
remote: Enumerating objects: 155, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 155 (delta 20), reused 13 (delta 13), pack-reused 126
Receiving objects: 100% (155/155), 29.41 MiB | 10.75 MiB/s, done.
Resolving deltas: 100% (46/46), done.
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 15 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 120882 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.2_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.2) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpa

In [ ]:
# input image - test.jpg

In [ ]:
%cd /content/3DDFA_V2
!rm -rf /content/stage/* /content/3DDFA_V2/crop_samples/img/* /content/3DDFA_V2/test/original/* /content/output/*
!cp /content/in/* /content/3DDFA_V2/test/original

!python dlib_kps.py
!python recrop_images.py -i data.pkl -j dataset.json

!cp -rf /content/3DDFA_V2/crop_samples/img/* /content/stage

# %cd /content/PanoHead
# !./gen_pti_script.sh

/content/3DDFA_V2
100% 1/1 [00:01<00:00,  1.85s/it]
results: 1


In [ ]:
%cd /content/PanoHead
!python projector_withseg.py --num-steps=300 --num-steps-pti=300 --outdir=/content/output --target_img=/content/stage --network=/content/PanoHead/models/easy-khair-180-gpc0.8-trans10-025000.pkl --idx 0

/content/PanoHead
Loading networks from "/content/PanoHead/models/easy-khair-180-gpc0.8-trans10-025000.pkl"...
projecting: [0] /content/stage/test.jpg
camera matrix: torch.Size([1, 25])
Computing W midpoint and stddev using 10000 samples...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
step:    1/300 mse: 0.18 perc: 0.53 w_norm: 0.00  noise: 0.00 
step:    2/300 mse: 0.18 perc: 0.59 w_norm: 0.00  noise: 0.00 
step:    3/300 mse: 0.19 perc: 0.58 w_norm: 0.00  noise: 0.00 
step:    4/300 mse: 0.17 perc: 0.49 w_norm: 0.00  noise: 0.00 
step:    5/300 mse: 0.16 perc: 0.49 w_norm: 0.00  noise: 0.00 
step:    6/300 mse: 0.18 perc: 0.49 w_norm: 0.00  noise: 0.00 
step:    7/300 mse: 0.14 perc: 0.46 w_norm: 0.00  noise: 0.00 
step:    8/300 mse: 0.15 perc: 0.50 w_norm: 0.00  noise: 0.00 
step:    9/300 mse: 0.14 perc: 0.48 w_norm: 0.01  noise: 0.00 
step:   10/300 mse: 0.17 perc: 0.47 w_norm: 0.01  noise: 0.00 
step:   11/300 mse: 0.

In [ ]:
%cd /content/PanoHead
!python gen_videos_proj_withseg.py --output=/content/output/easy-khair-180-gpc0.8-trans10-025000.pkl/0/PTI_render/pre.mp4 --latent=/content/output/easy-khair-180-gpc0.8-trans10-025000.pkl/0/projected_w.npz --trunc 0.7 --network /content/PanoHead/models/easy-khair-180-gpc0.8-trans10-025000.pkl --cfg Head

/content/PanoHead
Loading networks from "/content/PanoHead/models/easy-khair-180-gpc0.8-trans10-025000.pkl"...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
100% 240/240 [00:25<00:00,  9.39it/s]


In [ ]:
%cd /content/PanoHead
!python gen_videos_proj_withseg.py --output=/content/output/easy-khair-180-gpc0.8-trans10-025000.pkl/0/PTI_render/post.mp4 --latent=/content/output/easy-khair-180-gpc0.8-trans10-025000.pkl/0/projected_w.npz --trunc 0.7 --network /content/output/easy-khair-180-gpc0.8-trans10-025000.pkl/0/fintuned_generator.pkl --cfg Head

/content/PanoHead
Loading networks from "/content/output/easy-khair-180-gpc0.8-trans10-025000.pkl/0/fintuned_generator.pkl"...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
100% 240/240 [00:24<00:00,  9.83it/s]


In [ ]:
%cd /content/PanoHead
!python projector_withseg.py --shapes=True --num-steps=100 --num-steps-pti=100 --outdir=/content/output --target_img=/content/stage --network=/content/PanoHead/models/easy-khair-180-gpc0.8-trans10-025000.pkl --idx 0

/content/PanoHead
Loading networks from "/content/PanoHead/models/easy-khair-180-gpc0.8-trans10-025000.pkl"...
projecting: [0] /content/stage/test.jpg
camera matrix: torch.Size([1, 25])
Computing W midpoint and stddev using 10000 samples...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
step:    1/100 mse: 0.18 perc: 0.53 w_norm: 0.00  noise: 0.00 
step:    2/100 mse: 0.18 perc: 0.59 w_norm: 0.00  noise: 0.00 
step:    3/100 mse: 0.18 perc: 0.55 w_norm: 0.00  noise: 0.00 
step:    4/100 mse: 0.15 perc: 0.46 w_norm: 0.00  noise: 0.00 
step:    5/100 mse: 0.12 perc: 0.45 w_norm: 0.00  noise: 0.00 
step:    6/100 mse: 0.13 perc: 0.47 w_norm: 0.01  noise: 0.00 
step:    7/100 mse: 0.11 perc: 0.43 w_norm: 0.02  noise: 0.00 
step:    8/100 mse: 0.07 perc: 0.45 w_norm: 0.03  noise: 0.00 
step:    9/100 mse: 0.06 perc: 0.41 w_norm: 0.04  noise: 0.00 
step:   10/100 mse: 0.06 perc: 0.39 w_norm: 0.05  noise: 0.00 
step:   11/100 mse: 0.

## PLY to OBJ (optional)

In [ ]:
!pip install aspose-3d

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 MB 2.0 MB/s eta 0:00:00


In [ ]:
import aspose.threed as a3d

scene = a3d.Scene.from_file("/content/output/easy-khair-180-gpc0.8-trans10-025000.pkl/0/geometry.ply")
scene.save("/content/output/easy-khair-180-gpc0.8-trans10-025000.pkl/0/geometry.obj")